# Replicating [Yan et al. (2022)](https://arxiv.org/abs/2212.12372)

Our goal is to factor a number $N$ into two prime numbers $p$ and $q$. Yes, $N$ is selected so that this is always the case. Yan et al. (2022)'s claim is that, by using quantum computers to speed up classically-slow computations (specifically, finding smooth-relation pairs), then [Schnorr's classical factoring algorithm](https://eprint.iacr.org/2021/933) (which adopts a lattice-based approach to factoring) can be used to solve factoring with a "sublinear" number of qubits in the bit length of $N$.

The lattice used in Schnorr's algorithm is of dimension $n$, defined according to the bit length of $N$, which we'll call $m$. While Schnorr himself is a little coy on the exact formulation of his choice on $n$, Yan et al. (2022) suggest the dimension satisfies $n\sim2c\log N/\log\log N$, where $c$ is a lattice parameter close to $1$ that we'll meet in a moment.

Yan et al. (2022)'s method takes a relatively good [classical estimate solving the CVP](https://link.springer.com/article/10.1007/BF02579403), call it $b_{op}$, then uses QAOA to search in the unit hypercube on the lattice surrounding it, where "unit" is defined with consideration to the basis $D=[d_1,\dots,d_n]$ obtained by [lattice reduction](https://infoscience.epfl.ch/record/164484/files/nscan4.PDF).

So, all things considered, the implementation is structured as follows:

1. Firstly, we define the basis of the prime lattice $B_{n,c}\in\mathbb{R}^{(n+1)\times n}$, parameterised by the "precision parameter" $c$, and a target vector $t\in\mathbb{R}^{n+1}$ which embody the reduction of the prime factorisation problem to the CVP on a lattice, whose vectors are smooth-relation pairs (sr-pairs).
$$
    B_{n,c}=
\begin{pmatrix}
f(1)       & 0          & \cdots & 0      \\
0          & f(2)       & \cdots & 0      \\ 
\vdots     & \vdots     & \ddots & \vdots \\ 
0          & 0          & \cdots & f(n)   \\ 
N^c\ln p_1 & N^c\ln p_2 & \cdots & N^c\ln p_n
\end{pmatrix}
\phantom{\int}\text{and}\phantom{\int}
t=
\begin{pmatrix}
0 \\ 0 \\ \vdots \\ 0 \\ N^c\ln N
\end{pmatrix}
$$

2. Secondly, we perform lattice reduction (e.g. LLL) to obtain a reduced basis $D=[d_1,\dots,d_n]$ and corresponding Gram-Schmidt orthogonal basis $\tilde{D}=[\tilde{d}_1,\dots,\tilde{d}_n]$.

3. Thirdly, we obtain an approximate solution $b_{op}$ by rounding each of the Gram-Schmidt coefficients to give the coefficients $c_i=\lceil\mu_i\rfloor=\lceil\langle d_i,\tilde{d}_i\rangle/\langle\tilde{d}_i,\tilde{d}_i\rangle\rfloor$. That is,
$$
b_{op}=\sum_{i=1}^{n}c_id_i
$$

4. Then, we formulate an optimisation problem that looks for an even closer vector $v_{new}$ by floating each coefficient either $\pm1$ or $0$ (i.e. taking a unit step away or staying put for each lattice dimension). Hence, our optimisation problem is to find the set of floating variables $\{x_1,\dots,x_n\}$ that minimise the distance between $v_{new}$ and $t$;
$$
F(x_1,\dots,x_n)=\|t-v_{new}\|^2=\Bigg\|t-b_{op}+\sum_{i=1}^nx_id_i\Bigg\|^2.
$$

5. Next, this optimisation problem is mapped to a Hamiltonian by exchanging the floating variables $x_i$ for positive or negative Pauli-Z operators $\sigma_z^i$, whose measured eigenvalues correspond exactly with the classical optimisation problem. Denote $\hat{x}_i$ to be the Pauli-Z operator $\pm\sigma_z^i$ corresponding to up-floating or down-floating.
$$
H_P=\Bigg\|t-b_{op}+\sum_{i=1}^n\hat{x}_id_i\Bigg\|^2=\sum_{j=1}^{n+1}\Bigg|t_j-b^j_{op}+\sum_{i=1}^n\hat{x}_id_{i,j}\Bigg|^2.
$$

6. Jumping into the world of programming, we then build a [VQE](https://arxiv.org/abs/1304.3061) to minimise the expectation value of our Hamiltonian $H_P$ under the trial state $|\psi(\vec\theta)\rangle$ parameterised by $\vec\theta$. Yan et al. (2022) use QAOA, which is not convincing of its ability to provide quantum advantage.

## Problem Set-up

Decide on a number to be factored, given in binary.

In [9]:
import copy

# Imports as always...
import numpy as np
from fpylll import *
from copy import deepcopy

In [10]:
# Seed for random generation. The seed 99 gives the same B_{5,4} as in the paper.
np.random.seed(99)

In [11]:
# The integer to be factored.
p, q = 7919, 6133 
N = p * q

# What is the bit-length of the integer?
N_bit_length = N.bit_length()

# Convert to binary (drop the '0b' prefix).
N_binary = bin(N)[2:]

print(f'Integer to be factored: N = {N} (into the factors p = {p} and q = {q}).')
print(f'This has bit-length {N_bit_length}')

Integer to be factored: N = 48567227 (into the factors p = 7919 and q = 6133).
This has bit-length 26


The lattice dimension $m$ is given by $m=\lfloor ln/\log n\rfloor$, where $n=\lfloor\log N\rfloor$ and $l$ is a hyperparamter that Yan et al. (2022) keep to $l\in\{1,2\}$.

In [12]:
# The claimed lattice dimension to factor this integer is l * log N / log log N.
l = 1

n = np.log2(N)
m = (l * n) / np.log2(n)

# Round them, after the fact.
n, m = int(np.floor(n)), int(np.floor(m))

print(f'The claim is that we need only a lattice of dimension {m}.')

The claim is that we need only a lattice of dimension 5.


In [13]:
primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157, 163,167,173]

Constructing the prime lattice and target vector according to the integer to be factored. We do this according to appendix IV part A (page 13) of Yan et al. (2022). The hyperparameter $c$ is the "precision parameter".

In [14]:
c = 4

# Produce the random permutation for the diagonal.
f = np.random.permutation([(i + 1) // 2 for i in range(1, m + 1)])

# Create a zero matrix and add in the diagonal permutation.
B = np.zeros(shape=(m, m))
np.fill_diagonal(B, f)

# Create the extra final row and stick it on.
final_row = np.round(10 ** c * np.log(np.array(primes[:m])))
B = np.vstack((B, final_row))

# fpylll doesn't like numyp arrays, so convert it to a stnadard array.
B = [[int(b) for b in bs] for bs in B]

# Convert B to a matrix of integers (in fpylll's own type).
B = IntegerMatrix.from_matrix(B)
print(f'B = \n{B}')

# Define the target vector.
t = np.zeros(m + 1)
t[-1] = np.round(10 ** c * np.log(N))
t = tuple(t.astype(int))

# And again, if t could be a bunch of integers, that would be swell.
print(f'\nt = \n{t}')

B = 
[    2     0     0     0     0 ]
[    0     1     0     0     0 ]
[    0     0     3     0     0 ]
[    0     0     0     2     0 ]
[    0     0     0     0     1 ]
[ 6931 10986 16094 19459 23979 ]

t = 
(0, 0, 0, 0, 0, 176985)


## Babai's Algorithm: Finding an Approximate Solution $b_{op}$

Performing LLL lattice reduction to obtain a reduced basis $D$ and corresponding Gram-Schmidt orthogonal basis $\tilde{D}$.

Using the [fpylll](https://pypi.org/project/fpylll/) library, which is not supported on windows, so I am having to do this in the world's slowest virtual machine.

In [9]:
# LLL-reduction hyperparameter (using 0.75, according to Wikipedia)
delta = .75

D = deepcopy(B).transpose()
LLL.reduction(D, delta)
print(f'D (transposed) = \n{D}\n')

M = GSO.Mat(D, update=True)
w = M.babai(t)

A = IntegerMatrix(2 * D.nrows, D.ncols)
for i in range(D.nrows):
    for j in range(D.ncols):
        A[i, j] = D[i, j]

b = np.array(t)
for i in reversed(range(D.nrows)):
    for j in range(D.ncols):
        A[i + D.nrows, j] = int(b[j])
    b -= w[i] * np.array(D[i])

print("Which way is each coefficient rounded?")
M = GSO.Mat(A, update=True)
rounding_direction = []
for i in range(D.nrows):
    mu = M.get_mu(i + D.nrows, i)
    print(f'mu={mu} c={w[i]}')
    rounding_direction.append(w[i] > mu)
  
b_op = np.array(D.multiply_left(w))
residual_vector = np.array(t) - b_op
print(f'\nb_op = \n{b_op}\n')
print(f'Hence, the residual vector is \n{residual_vector}\n')
print(f'This has a distance of {round(np.linalg.norm(residual_vector), 3)} to the target vector.')

D (transposed) = 
[  6 -4  6  4 -2  -3 ]
[ -8 -3  6 -2  2   5 ]
[  2 11  3  0 -6  -3 ]
[ -4 -5  0 12 -2   4 ]
[ -4 -3 -3  4  1 -17 ]

Which way is each coefficient rounded?
mu=-3092.4957264957266 c=-3092
mu=-354.46685552407934 c=-354
mu=-1837.4759673530396 c=-1837
mu=3882.501940752412 c=3883
mu=-8731.560681864921 c=-8732

b_op = 
[     2      4      9      8      0 176993]

Hence, the residual vector is 
[-2 -4 -9 -8  0 -8]

This has a distance of 15.133 to the target vector.


In [10]:
# Reformatting after this step to make subsequent steps easier.
def integer_matrix_to_numpy(M):
  m, n = M.nrows, M.ncols
  D = np.zeros((m, n), dtype=int)
  M.to_matrix(D)
  return D

B = integer_matrix_to_numpy(B)
D = integer_matrix_to_numpy(D.transpose())
t = np.array(t)
rounding_direction = tuple(rounding_direction)

## The Problem Hamiltonian

The new vector $v_{new}$ to be optimally found is parameterised on $b_{op}$ in the following way:
$$
v_{new}=b_{op}+\sum_{i=1}^nx_id_i=\sum_{i=1}^n(c_i+x_i)d_i
$$

Naively, we may choose $x_i\in\{\pm1,0\}$ to step a unit amount in direction $d_i$ (or not move at all), which thus paints out the unit hypercube around $b_{op}=\sum_{i=1}^nc_id_i$ as our search space. However, we need not be this naive!

During the finding of $b_{op}$, the coefficients $c_i$ were found by rounding the Gram-Schmidt coefficients $\mu_i$ to the nearest integer. Hence, $b_{op}$ either overestimates or underestimates the $d_i$-th component. So when we are nudging $b_{op}$ around to yield $v_{new}$, there is no need to consider *further* over/under-estimation! That is, if $c_i$ is obtained through rounding $\mu_i$ upward, then using $x_i$ should not be $+1$ as that would overestimate *again* (and likewise for the rounding down case).

Consider the following:
$$
v_{new}=\sum_{i=1}^nc_id_i+\text{sign}(\mu_i-c_i)x_id_i=\sum_{i=1}^nc_id_i+\lceil \mu_i-c_i\rceil x_id_i
$$

Now, we may have $x_i\in\{0,1\}$ (giving us a pure QUBO formulation, as desired) which correspond to $n$ decisions about whether each dimension of the approximate solution $d_i$ was rounded the wrong way ($x_i=1$ indicates that $v_{new}$ tries rounding the other way, and $x_i=0$ indicates that $v_{new}$ agrees with the rounding of $b_{op}$ and nothing need be done).

Since $\text{sign}(\mu_i-c_i)=\lceil\mu_i-c_i\rceil=\pm1$, this additional coefficient gives us whether $b_{op}$ over- or under-estimated, thus allows us to make the restriction on which direction $v_{new}$ is able to 'step'. We have that $|\mu_i-c_i|<1$, thus $\lceil\mu_i-c_i\rceil=+1$ if $c_i$ is obtained by rounding down $\mu_i$ and $-1$ if by rounding up.

By keeping $x_i\in\{0,1\}$, we enforce that $v_{new}$ either step in the opposing direction to the rounding (i.e. round the other way on the coefficient $\mu_i$) or do nothing. If $x_i=1$, then $v_{vew}$'s $i$-th coefficient will be $\lceil\mu_i-c_i\rceil=\pm1$ step from $b_{op}$'s coefficient, corresponding to deciding to round $\mu_i$ to the *second*-nearest integer.

To complete the thought, the optimisation problem can now be formulated as a strictly binary optimisation problem;
$$
F(x_1,\dots,x_n)=\|t-v_{new}\|^2=\Bigg\|t-b_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil x_id_i\Bigg\|^2.
$$
and hence the Hamiltonian is obtained through a singular mapping;
$$
H_P=\Bigg\|t-b_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil\hat{x}_id_i\Bigg\|^2=\sum_{j=1}^{n+1}\Bigg|t_j-b^j_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil\hat{x}_id_{i,j}\Bigg|^2\phantom{\int}\text{with}\phantom{\int}\hat{x}_i=\frac{I-\sigma_z^i}{2}
$$

Note: the above ignores that $\mu_i$ may well have been negative, so each $\lceil\mu_i-c_i\rceil$ should really be $\lceil|\mu_i|-|c_i|\rceil$.

Let's define the Hamiltonian using `cirq`, which lets us do $I-\sigma_z^i$ very easily with `cirq.I(i) + -cirq.Z(i)`, where `i` refers to the $i$-th qubit and thus the tensor product is implicit. 

In [11]:
# More imports...
import cirq

In [15]:
# We have kept track of whether we rounded up, so convert this to +/-1 indications of whether each c_i is a rounding up or down.
# The direction v_new is allowed to step in each d_i is the opposite of this.
step_signs = - (np.array(rounding_direction).astype(int) * 2 - 1)

# Define the circuit.
circuit = cirq.LineQubit.range(D.shape[0])

# Add the appropriate operator to each qubit.
operators = []
for i, sign in zip(circuit, step_signs):
    operator = sign * ((cirq.I(i) + -cirq.Z(i)) / 2)
    operators.append(operator)
    
# Define the Hamiltonian.
H = cirq.PauliSum()
for j in range(D.shape[0]):
    h = residual_vector[j] 
    for i in range(D.shape[1]):
        print(f'{int(D[j,i])}x_{i}')
        h -= operators[i] * D[j, i]
    print(f'{residual_vector[j]}\n')
    print(f'{h}\n')
    H += h ** 2
    
print(f'H = \n{H}')

6x_0
-8x_1
2x_2
-4x_3
-4x_4
-2

-3.0*Z(q(0))-2.0*I+4.0*Z(q(1))-1.0*Z(q(2))+2.0*Z(q(3))-2.0*Z(q(4))

-4x_0
-3x_1
11x_2
-5x_3
-3x_4
-4

2.0*Z(q(0))-3.0*I+1.5*Z(q(1))-5.5*Z(q(2))+2.5*Z(q(3))-1.5*Z(q(4))

6x_0
6x_1
3x_2
0x_3
-3x_4
-9

-3.0*Z(q(0))-3.0*Z(q(1))-1.5*Z(q(2))-1.5*Z(q(4))

4x_0
-2x_1
0x_2
12x_3
4x_4
-8

-2.0*Z(q(0))-3.0*I+1.0*Z(q(1))-6.0*Z(q(3))+2.0*Z(q(4))

-2x_0
2x_1
-6x_2
-2x_3
1x_4
0

1.0*Z(q(0))-1.0*Z(q(1))+3.0*Z(q(2))-4.5*I+1.0*Z(q(3))+0.5*Z(q(4))

-3x_0
5x_1
-3x_2
4x_3
-17x_4
-8

1.5*Z(q(0))+2.0*I-2.5*Z(q(1))+1.5*Z(q(2))-2.0*Z(q(3))-8.5*Z(q(4))

H = 
292.0*I+3.5*Z(q(0))*Z(q(2))+18.0*Z(q(0))*Z(q(3))-17.5*Z(q(0))*Z(q(4))-32.0*Z(q(1))+16.0*Z(q(2))-4.0*Z(q(3))-33.5*Z(q(4))-29.0*Z(q(1))*Z(q(2))+19.5*Z(q(1))*Z(q(3))+34.0*Z(q(1))*Z(q(4))-31.5*Z(q(2))*Z(q(3))+2.5*Z(q(2))*Z(q(4))-4.5*Z(q(3))*Z(q(4))+9.0*Z(q(0))-13.5*Z(q(0))*Z(q(1))


In [12]:
# Pretty printing the Hamiltonian.
string_H = str(H)
string_H = string_H.replace('+', '\n+')
string_H = string_H.replace('-', '\n-')
string_H = string_H.replace('*', ' * ')
print(string_H)

292.000 * I
+3.500 * Z(q(0)) * Z(q(2))
+18.000 * Z(q(0)) * Z(q(3))
-17.500 * Z(q(0)) * Z(q(4))
-32.000 * Z(q(1))
+16.000 * Z(q(2))
-4.000 * Z(q(3))
-33.500 * Z(q(4))
-29.000 * Z(q(1)) * Z(q(2))
+19.500 * Z(q(1)) * Z(q(3))
+34.000 * Z(q(1)) * Z(q(4))
-31.500 * Z(q(2)) * Z(q(3))
+2.500 * Z(q(2)) * Z(q(4))
-4.500 * Z(q(3)) * Z(q(4))
+9.000 * Z(q(0))
-13.500 * Z(q(0)) * Z(q(1))


## Sampling Low-energy Eigenstates of the Hamiltonian by QAOA

[This](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising) seems like a particularly relevant article from Google Quantum AI, which discusses QAOA for the Ising model (which our Hamiltonian is defined as) using Cirq.

Specifically, we have the binary optimisation problem $F(z_1,\dots,z_n)$, where each $z_i$ corresponds to the measurement outcome (i.e. an eigenvalue) of the Pauli-$Z$ operator on the $i$-th qubit (we're calling then $z_i$ rather than $x_i$ now because of this Pauli-$Z$ correlation -- it's just nice).

First, we prepare $n$ qubits in an equal superposition of all possible $z$ assignments; 

$$
H^{\otimes n}|0^n\rangle=\frac{1}{2^{n/2}}\sum_{z\in\{0,1\}^n}|z\rangle
$$

Now, our goal is to amplify the amplitdues of the $|z\rangle$ for which $F(z)$ is small, and suppress the amplitudes of those for which $F(z)$ is large. This will allow us to measure better choices of $z$ with higher probability.

Next, we act with the unitary $U(\gamma,F)=e^{i\pi\gamma F(Z)/2}$, where $\gamma$ is variational parameter, and $F(Z)$ denotes that we have exchanged the $z_i$ for Pauli-$Z$ operators, thus turning $F(z)$ (a real number) into $F(Z)$ (a matrix whose diagonal entries represent the possible values of $F(z)$). This sets the coefficients in the sum to be complex phases depending on $F$.

Following this, we act with the unitary operator $U(\beta,B)=e^{i\pi\beta B/2}$ (the "mixing" operation), where $\beta$ is another variational parameter, and $B=\sum_{i=1}^nX_i$ is the sum of Pauli-$X$ operator acting on each qubit. Each of these Pauli-$X$ operators commute, since they each apply to a different qubit, so we can rewrite this unitary as
$$
U(\beta,B)=\prod_{i=1}^ne^{i\pi\beta X_i/2}
$$

Since this operation is *not* diagonal in the computational basis (unlike the previous), there will be constructive and destructive interference, hopefully leading to enhancement of states corresponding to small values of $F$.

These two steps are repeatedly applied $p\geq1$ times, where $p$ here denotes the depth of the circuit. We allow $\gamma$ and $\beta$ to be chosen afresh each time (otherwise there wouldn't be much point in repeating it), giving a circuit that leaves our qubits in the state
$$
|\gamma,\beta\rangle=U(\beta_p,B)U(\gamma_p,F)\cdots U(\beta_1,B)U(\gamma_1,F)$
$$

It is our job to select $\gamma$ and $\beta$ so that the expectation value $F(\gamma,\beta)=\langle\gamma,\beta|F(Z)|\gamma,\beta\rangle$ is minimised so that our measurement of $|\gamma,\beta\rangle$ in the computational basis gives a good solution $z$ for $F(z)$ with high probability.

In our case, we have defined $F(Z)=H_P$.

### Defining the gamma unitary operations

Our $H_P$ was formulated as a two-dimensional Ising model, hence it is of the form $\sum_ih_iZ_i+\sum_{i,j}J_{i,j}Z_iZ_j$, where $h_i$ and $J_{i,j}$ are determined by the coefficients of the primary and quadratic terms of the QUBO problem.

Because $H_P$ is of this form, $e^{-i\gamma H_p}$ may be implemented as a product of the individual single qubit terms $Z_i^{\gamma h_i}$ and two qubit terms $(ZZ)_{ij}^{\gamma J_{i,j}}$.

In [13]:
# More imports...
import sympy
import qsimcirq
from scipy.optimize import minimize

In [19]:
# Building the circuit.
p = 10

# Get references to the qubits used in the Hamiltonian's definition.
qubits = H.qubits

# Sub-circuit implementing the gamma unitary operator for the i-th layer.
def generate_gamma_operator(i):
    # Define the gamma symbol (as a placeholder).
    gamma = sympy.Symbol(f'gamma_{i}')
    
    # Define the Pauli operators as dense Pauli strings (for recognising terms).
    Z = cirq.DensePauliString('Z')
    ZZ = cirq.DensePauliString('ZZ')
    I = cirq.DensePauliString('')
    
    # Consider each term in the Hamiltonian.
    for term in H:
        # Get the term's operator.
        term_operator = term.with_coefficient(1).gate
        
        # Determine which circuit translation to make, based on which operator this term has.
        if term_operator == Z:
            yield cirq.Z(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == ZZ:
            yield cirq.ZZ(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == I:
            yield []
            
        # If the term's operator is unrecognised, the Hamiltonian isn't quite right.
        else:
            raise Exception(f'Unrecognised term in H: {term}')
        
# Sub-circuit implementing the beta unitary ("mixing") operator for the i-th layer.
def generate_beta_operator(i):
    # Define the beta symbol (as a placeholder).
    beta = sympy.Symbol(f'beta_{i}')
    
    return [cirq.X(q) ** beta for q in qubits]

# Generate the circuit for QAOA.
qaoa_circuit = cirq.Circuit(
    # Preparation of uniform superposition.
    cirq.H.on_each(*qubits),
    
    # p layers of repeated U(gamma, H) and U(beta, B) operations.
    [(generate_gamma_operator(i), cirq.Moment(generate_beta_operator(i)),) for i in range(p)],
)

display(qaoa_circuit)

┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐                                                       ┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌────────────────────────────────────┐                       ┌───────────────────────────────────┐
0: ───H────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_0)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_0────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_1)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_1────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_2)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_2────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_3)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_3────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_4)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_4────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_5)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_5────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_6)─────────────────────────────────────────────ZZ───────────────────────────────────────X^beta_6────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ──────────────────────────────────────Z^(9.0*gamma_7)───────────────────

In [20]:
# Finding the optimal parameters for the circuit.

# Get the parameters and observables out of the circuit.
parameter_names = sorted(cirq.parameter_names(qaoa_circuit))
observables = [term.with_coefficient(1) for term in H]

# Define the function to minimise.
def func_to_minimise(x):
    # Define a "parameter resolver" object, which we can use to assign values to parameters.
    parameter_resolver = cirq.ParamResolver(
        # Map each parameter to a value (i.e. assign the circuit's parameters).
        {parameter: value for parameter, value in zip(parameter_names, x)}
    )
    
    # Define a circuit simulator.
    simulator = qsimcirq.QSimSimulator(
        # Pass in some options for the simulator object -- this needs another object, it seems.
        qsimcirq.QSimOptions(
             # Cannot use GPU without compiling qsim locally -- maybe another time.
             use_gpu=False,
             # My PC has 8 cores, so set 8 threads.
             cpu_threads=8,
             verbosity=0
        )
    )
    
    # Simulate the expectation value.
    expectation = simulator.simulate_expectation_values(
        program=qaoa_circuit, observables=observables, param_resolver=parameter_resolver
    )
    
    # Compute the return.
    return sum(term.coefficient * value for term, value in zip(H, expectation)).real

# Let our initial guess be all zeros -- we don't know any better.
x_initial = np.asarray([.0] * len(parameter_names))

# Find the parameters values that minimise the expectation value.
optimal_parameter_values = minimize(func_to_minimise, x_initial, method='BFGS')
display(optimal_parameter_values)

# Stick the parameter values in a parameter resolver object.
optimal_parameters = cirq.ParamResolver(
    {parameter: value for parameter, value in zip(parameter_names, optimal_parameter_values.x)}
)
print(f'\nOptimal parameters: {optimal_parameters}')

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 292.0
        x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
 hess_inv: [[1 0 ... 0 0]
            [0 1 ... 0 0]
            ...
            [0 0 ... 1 0]
            [0 0 ... 0 1]]
     nfev: 21
     njev: 1


Optimal parameters: cirq.ParamResolver({'beta_0': 0.0, 'beta_1': 0.0, 'beta_2': 0.0, 'beta_3': 0.0, 'beta_4': 0.0, 'beta_5': 0.0, 'beta_6': 0.0, 'beta_7': 0.0, 'beta_8': 0.0, 'beta_9': 0.0, 'gamma_0': 0.0, 'gamma_1': 0.0, 'gamma_2': 0.0, 'gamma_3': 0.0, 'gamma_4': 0.0, 'gamma_5': 0.0, 'gamma_6': 0.0, 'gamma_7': 0.0, 'gamma_8': 0.0, 'gamma_9': 0.0})


In [2]:
# Sample solutions several times from the circuit with these optimal parameters.
runs = 10000

# Define a circuit simulator.
simulator = qsimcirq.QSimSimulator(
    qsimcirq.QSimOptions(
         use_gpu=False,
         cpu_threads=8,
         verbosity=0
    )
)

# Add a measurement operator at the very end (across all qubits).
qaoa_circuit_with_measurement = qaoa_circuit + cirq.Circuit(cirq.measure(H.qubits, key='m'))

# Run the circuit a bunch.
measurement_outcomes = simulator.run(
    qaoa_circuit_with_measurement, param_resolver=optimal_parameters, repetitions=runs
)

# Save the results into a histogram-like dictionary.
results = measurement_outcomes.histogram(key='m')
print(f'Top three measurements (solution, frequency): {results.most_common(3)}')

# Pull out the solutions and their frequency in the simulation measurements, in descending order.
solutions, frequencies = zip(*results.most_common(len(results)))

NameError: name 'qsimcirq' is not defined

### Converting QAOA solutions to lattice vectors

In [22]:
# Helper function to convert integers to n-bit binary.
def integer_to_binary_n(x):
    assert x < 2 ** n, f'Cannot convert {x} to {n}-bit binary.'
 
    # Convert to a binary string.
    x_as_bin = bin(x)[2:]
    
    # Prefix with 0s to bring to length n.
    if len(x_as_bin) != n:
        prefix = '0' * (n - len(x_as_bin))
        return prefix + x_as_bin
    else:
        return x_as_bin

In [33]:
D_temp = IntegerMatrix(D.shape[1], D.shape[0])
for i in range(D.shape[1]):
    for j in range(D.shape[0]):
        D_temp[i, j] = int(D[j, i])
        
print(D_temp)

[  6 -4  6  4 -2  -3 ]
[ -8 -3  6 -2  2   5 ]
[  2 11  3  0 -6  -3 ]
[ -4 -5  0 12 -2   4 ]
[ -4 -3 -3  4  1 -17 ]


In [1]:
# Convert an integer solution to a lattice vector.
def solution_to_vector(solution):
    # Convert the solution to a binary string of assignments.
    assignments = integer_to_binary_n(solution)
    
    # Use the assignments to step in each reduced basis from b_op.
    w_new = np.zeros(shape=(n, 1))
    for i, sign in enumerate(step_signs):
        w_new[i] = w[i] + (int(assignments[i]) * sign)
        
    # Yield the new vector.
    return np.array(D_temp.multiply_left(w_new))
        
solution_to_vector(solutions[0])

NameError: name 'solutions' is not defined